In [1]:
import math

import numpy as np
import pandas as pd
import datetime as dt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from RandomForestForTimeSeriesClassifier import RandomForestForTimeSeriesClassifier

In [2]:
dataset = pd.read_csv('../datasets/sp_500_stocks/sp500_stocks.csv', nrows=40000)

In [3]:
dataset = dataset.dropna()
dataset

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
22524,2010-01-04,AMD,9.700000,9.700000,9.900000,9.680000,9.790000,18748700.0
22525,2010-01-05,AMD,9.710000,9.710000,9.900000,9.680000,9.710000,22145700.0
22526,2010-01-06,AMD,9.570000,9.570000,9.760000,9.550000,9.680000,18643400.0
22527,2010-01-07,AMD,9.470000,9.470000,9.550000,9.180000,9.510000,26806800.0
22528,2010-01-08,AMD,9.430000,9.430000,9.470000,9.290000,9.370000,13752800.0
...,...,...,...,...,...,...,...,...
39995,2019-10-04,APD,190.052368,213.880005,214.899994,211.580002,212.130005,770700.0
39996,2019-10-07,APD,190.123444,213.960007,215.899994,212.500000,213.330002,727800.0
39997,2019-10-08,APD,184.614120,207.759995,213.350006,207.660004,213.350006,1025300.0
39998,2019-10-09,APD,188.150772,211.740005,212.660004,208.820007,210.119995,810100.0


In [4]:
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Date'] = dataset['Date'].map(dt.datetime.toordinal)
#dataset['Date'] = pd.to_float(dataset['Date'])
# dataset['Date'] = datetime.datetime.strptime(dataset['Date'], '%Y-%m-%d')
# dataset['Date'] = float(time.mktime(dataset['Date']))
dataset['Symbol'] = pd.factorize(dataset.Symbol)[0]
dataset['Adj Close'] = dataset['Adj Close'] * 100
dataset['Adj Close'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Close'] = dataset['Close'] * 100
dataset['Close'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['High'] = dataset['High'] * 100
dataset['High'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Low'] = dataset['Low'] * 100
dataset['Low'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Open'] = dataset['Open'] * 100
dataset['Open'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Volume'] = np.floor(pd.to_numeric(dataset['Volume'], errors='coerce')).astype('Int64')
dataset

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
22524,733776,0,969,969,969,969,969,18748700
22525,733777,0,971,971,971,971,971,22145700
22526,733778,0,956,956,956,956,956,18643400
22527,733779,0,947,947,947,947,947,26806800
22528,733780,0,943,943,943,943,943,13752800
...,...,...,...,...,...,...,...,...
39995,737336,1,19005,19005,19005,19005,19005,770700
39996,737339,1,19012,19012,19012,19012,19012,727800
39997,737340,1,18461,18461,18461,18461,18461,1025300
39998,737341,1,18815,18815,18815,18815,18815,810100


In [5]:
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(dataset)
# normalized_dataset = pd.DataFrame(x_scaled)
#
# normalized_dataset

In [6]:
dataset = dataset.sort_values("Date")
dataset

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
22524,733776,0,969,969,969,969,969,18748700
37540,733776,1,5312,5312,5312,5312,5312,1131483
37541,733777,1,5268,5268,5268,5268,5268,790103
22525,733777,0,971,971,971,971,971,22145700
37542,733778,1,5225,5225,5225,5225,5225,1248663
...,...,...,...,...,...,...,...,...
26273,739215,0,14113,14113,14113,14113,14113,30923100
26274,739216,0,13772,13772,13772,13772,13772,32092400
26275,739217,0,13624,13624,13624,13624,13624,30175300
26276,739219,0,13717,13717,13717,13717,13717,16085700


In [7]:
trainPercent = 80
trainSamples = len(dataset.index) // trainPercent
testSamples = len(dataset.index) - trainSamples

In [8]:
aux = dataset.copy()

X = aux.pop("Date")

# Obtener los valores de la variable dependiente (objetivo)
Y = aux.copy()
dataset = aux

X_train = X.iloc[:trainSamples]
Y_train = Y.iloc[:trainSamples]
X_test = X.iloc[-testSamples:]
Y_test = Y.iloc[-testSamples:]

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=1)
#
X_train = X_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)


In [21]:
basicRF = RandomForestClassifier(n_jobs=10, n_estimators=10000)

In [25]:
basicRF.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=10000, n_jobs=10)

In [26]:
# Realizar predicciones sobre los datos de prueba
y_pred = basicRF.predict(X_test)

In [27]:
from sklearn import metrics

print("RMSE:")
print(np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

RMSE:
16651471.174244644


In [13]:
count_row = dataset.shape[0]
#block_size = int(math.sqrt(count_row))
block_size = int(count_row // 100)

In [14]:
rfts = RandomForestForTimeSeriesClassifier(n_jobs=10, n_estimators=10000, block_size=block_size)

In [15]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [16]:
rfts.fit(X_train, Y_train)

RandomForestForTimeSeriesClassifier(block_size=62, n_estimators=10000,
                                    n_jobs=10)

In [17]:
y_pred = rfts.predict(X_test)

In [18]:
print("RMSE:")
print(np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

RMSE:
16612930.817185381


In [19]:
rmse = dict()
for i in range(10, 100):
    block_size = int(count_row // i)
    rfts = RandomForestForTimeSeriesClassifier(n_jobs=10, n_estimators=10000, block_size=block_size)
    rfts.fit(X_train, Y_train)
    y_pred = rfts.predict(X_test)
    print(f"RMSE: ({i}, {block_size})")
    print(np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
    rmse[i] = (block_size, np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

RMSE: (10, 621)
16858282.744228303
RMSE: (11, 564)
16867885.645924088
RMSE: (12, 517)
16837378.076494474
RMSE: (13, 478)
16837378.07696085
RMSE: (14, 443)
15146811.401883129
RMSE: (15, 414)
14033238.216065405
RMSE: (16, 388)
13426234.294842191
RMSE: (17, 365)
16617648.173180688
RMSE: (18, 345)
16867885.645924088
RMSE: (19, 327)
16822208.649335705
RMSE: (20, 310)
13336158.623345215
RMSE: (21, 295)
16375206.807827234
RMSE: (22, 282)
13426234.294257324
RMSE: (23, 270)
13551047.820805011
RMSE: (24, 258)
14368744.136154149
RMSE: (25, 248)
13523020.005897429
RMSE: (26, 239)
17043801.214292757
RMSE: (27, 230)
16348728.752162874
RMSE: (28, 221)
16867885.645924088
RMSE: (29, 214)
16837378.076494474
RMSE: (30, 207)
16686378.002389058
RMSE: (31, 200)
14305500.278200788
RMSE: (32, 194)
13642621.463149294
RMSE: (33, 188)
16838428.485513143
RMSE: (34, 182)
16858282.743762504
RMSE: (35, 177)
14011399.370118273
RMSE: (36, 172)
16918480.752085302
RMSE: (37, 167)
16889380.69919109
RMSE: (38, 163)
138474

In [20]:
rmse

{10: (621, np.float64(16858282.744228303)),
 11: (564, np.float64(16867885.645924088)),
 12: (517, np.float64(16837378.076494474)),
 13: (478, np.float64(16837378.07696085)),
 14: (443, np.float64(15146811.401883129)),
 15: (414, np.float64(14033238.216065405)),
 16: (388, np.float64(13426234.294842191)),
 17: (365, np.float64(16617648.173180688)),
 18: (345, np.float64(16867885.645924088)),
 19: (327, np.float64(16822208.649335705)),
 20: (310, np.float64(13336158.623345215)),
 21: (295, np.float64(16375206.807827234)),
 22: (282, np.float64(13426234.294257324)),
 23: (270, np.float64(13551047.820805011)),
 24: (258, np.float64(14368744.136154149)),
 25: (248, np.float64(13523020.005897429)),
 26: (239, np.float64(17043801.214292757)),
 27: (230, np.float64(16348728.752162874)),
 28: (221, np.float64(16867885.645924088)),
 29: (214, np.float64(16837378.076494474)),
 30: (207, np.float64(16686378.002389058)),
 31: (200, np.float64(14305500.278200788)),
 32: (194, np.float64(13642621.46